In [1]:
import numpy as np
import pandas as pd 
import jieba 
from collections import Counter
jieba.add_word('caonidaye')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hebo\AppData\Local\Temp\jieba.cache
Loading model cost 0.830 seconds.
Prefix dict has been built succesfully.


### 以下做了两个操作：
    第一：选择那些标签名存在于我们的商品描述之中的条目；
    第二：如果满足条件一，重新分词，将label作为自定义字典；

In [2]:
X_1 = open('.././input/train_X.txt','r',encoding = 'utf8').readlines()
Y_1 = open('.././input/train_Y.txt','r',encoding = 'utf8').readlines()
X = []
Y = []
X_del = []
Y_del = []
for i in range(len(Y_1)):
    X_1[i] = X_1[i].strip('\n')
    Y_1[i] = Y_1[i].strip('\n')
    description = X_1[i].replace(' ','')
    if Y_1[i] in description and Y_1[i] != '':
        drop_Y = description.replace(Y_1[i],'caonidaye') # 将描述里面的关键词抽取出来
        drop_cut = jieba.lcut(drop_Y) #对剩余的商品描述分词
        s = ''
        for j in range(len(drop_cut)):
            if drop_cut[j] == 'caonidaye':
                drop_cut[j] = Y_1[i]
            s = s + drop_cut[j] + ' '
        X.append(s)
        Y.append(Y_1[i])
    else:
        X_del.append(X_1[i])
        Y_del.append(Y_1[i])
#筛选词不在商品里面的 筛除 116358 - 104252 = 12096 条数据
#X_del用来存放这些垃圾

len(Counter(Y).keys()) = 18480

In [3]:
label_dict = {} # 创建规则字典
for i in range(len(X)):
    replace = False 
    label_cadi = [] #找出备选标签
    for word in X[i].split():
        if word in Y:
            label_cadi.append(word)
    #将备选标添加到label_cadi里面
    if Y[i] not in label_dict.keys():
        label_dict[Y[i]] = []
    for word_1 in label_cadi:
        for word_2 in label_cadi:
            if word_1 != word_2:
                if word_1 in label_dict.keys():
                    if word_2 in label_dict[word_1]:
                        Y[i] = word_1
                        replace = True
                        break
                #如果规则字典里面有，将此时Y值更新为规则词典
        if replace == True:
            break
        #如果已经代替，跳出循环
    if replace == True:
        continue
    #如果已经代替。继续下一条商品描述
    else:
        #label_cadi.remove(Y_refresh[i])
        #label_dict[Y_refresh[i]].extend(label_cadi)
        for word in label_cadi:
            if word!= Y[i]:
                label_dict[Y[i]].append(word)
     #如果没有代替的话，创建该规则字典   
#可以考虑加入if Y_refresh.count(Y_refresh[i]) < 10  直接break

##### len(Counter(Y).keys()) = 13397，通过构造数据字典，label的种类变为13398个

##### 下面做的任务就是统计数目在50-500 之间的类目，而且，分词后词数要大于等于5

In [4]:
Y_dic = Counter(Y)
X_new = [] 
Y_new = []
for i in range(len(X)):
    if Y_dic[Y[i]] >= 30 and Y_dic[Y[i]] <= 500:
        X_new.append(X[i])
        Y_new.append(Y[i])

In [58]:
frame = pd.concat([pd.Series(X_new).to_frame(),pd.Series(Y_new).to_frame()],axis = 1)
frame.columns = ['商品描述','标签']

##### 将以上处理之后的数据保存到文件夹中

In [59]:
with open('two_X_new.txt','w',encoding = 'utf8') as X_op:
    for i in range(len(X_new)):
        X_op.write(X_new[i] + '\n')
with open('two_Y_new.txt','w',encoding = 'utf8') as Y_op:
    for i in range(len(Y_new)):
        Y_op.write(Y_new[i] + '\n')

##### 使用word2cev分

In [60]:
from gensim.models import word2vec
sentences = word2vec.Text8Corpus('two_X_new.txt')
model = word2vec.Word2Vec(sentences,size = 50,window = 4 ,min_count = 3)
model.wv.save_word2vec_format('two.zh.text.vector',binary =False)

c:\python3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


##### 下面做的是将每条商品描述的word2vec相加求平均，保存到文件中

In [61]:
output = open('two_word2cev_60dim.txt','w',encoding = 'utf8')
misnum = 0
dim = 50
for line in range(len(X_new)):
    sum = np.zeros((1,dim))
    words = X_new[line].split()
    num = 1
    for word in words:
        try:
            temp = model[word]
        except:
            misnum +=1
        else:
            temp = temp.reshapeape((1,dim))
            sum += temp
            num = num+1
    if num > 1:
        num -= 1
    sum /= num
    for j in range(dim):
        output.write('%.4f'%(sum[0][j]) + ' ')
    output.write('\n')
output.flush() #刷新缓冲区
output.close()

c:\python3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [62]:
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd 

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


##### 调用我们的X和Y，以0.8和0.2的比例划分训练集和数据集

In [63]:
X = open('two_word2cev_60dim.txt','r',encoding = 'utf8')
Y = open('two_Y_new.txt','r',encoding = 'utf8')
X_lines = X.readlines()
Y_lines = Y.readlines()
for i in range(len(X_lines)):
    X_lines[i] = X_lines[i].strip('\n')
    X_lines[i] = list(map(float,X_lines[i].split()))
    Y_lines[i] = Y_lines[i].strip('\n')
label = LabelEncoder()
Y_num = label.fit_transform(Y_lines)
X_train,X_test,Y_train,Y_test = train_test_split(X_lines,Y_num,test_size = 0.2)

In [64]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train,Y_train)
y_pre = clf.predict(X_test)
result = np.mean(y_pre==Y_test)

In [65]:
result

0.46852160533401715

In [66]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
y_pre = clf.predict(X_test)
result = np.mean(y_pre == Y_test)

In [67]:
result

0.4862161815617387

In [68]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(np.array(X_train),Y_train)
y_pre = clf.predict(X_test)
result = np.mean(y_pre == Y_test)

c:\python3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [69]:
result

0.5354532632388768

In [87]:
len(X_lines)

38994

In [1]:

sum = np.zeros((1,50))

NameError: name 'np' is not defined

In [ ]:
sum